In [ ]:
# Caso esteja usando Google Colab, descomente a linha abaixo

# !pip install basedosdados

# Caso esteja utilizando o ambiente local, descomente as linhas abaixo

# import subprocess
# subprocess.run('pip install basedosdados', shell=True)

# Desafio Escritório de Dados

O desafio consiste em responder uma série de perguntas a partir da análise de 3 tabelas de dados. Como o programa faz download dos dados e guarda em memória, o mais eficiente seria realizar as analises em sql e só rodar o mínimo para diminuir o volume de dados.

Entretanto, na descrição é solicitado a utilização da biblioteca pandas. Dessa forma, iremos carregar as tabelas em memória para que possamos realizar as manipulações em pandas.

O único corte que será feito é selecionar apenas dados entre 01/01/2022 e 31/12/2023 da tabela de chamados.

In [2]:
#@title Configuring databases 💽
import basedosdados as bd
import pandas as pd

# selecionando dados entre 01/01/2022 e 31/12/2023 para diminuir a base para download
query_chamados = """
SELECT
  *
FROM datario.administracao_servicos_publicos.chamado_1746
WHERE data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
"""

query_bairros = """
SELECT
  *
FROM datario.dados_mestres.bairro
"""

query_eventos = """
SELECT
  *
FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos
"""

chamados = bd.read_sql(query_chamados, billing_project_id='emp-data-rio')


Downloading: 100%|██████████|


In [3]:
bairros = bd.read_sql(query_bairros, billing_project_id='emp-data-rio')
eventos = bd.read_sql(query_eventos, billing_project_id='emp-data-rio')

eventos['data_inicial'] = pd.to_datetime(eventos['data_inicial'])
eventos['data_final'] = pd.to_datetime(eventos['data_final'])

Downloading: 100%|██████████|
Downloading: 100%|██████████|


# Parte 1 - Localização de chamados do 1746

In [4]:
#@title Quantos chamados foram abertos no dia 01/04/2023?

chamados.loc[chamados['data_inicio'].astype(str).str[:10] == '2023-04-01'].shape[0]

73

In [5]:
#@title Qual o tipo de chamado que teve mais reclamações no dia 01/04/2023?

chamados.loc[chamados['data_inicio'].astype(str).str[:10] == '2023-04-01']\
  .groupby('tipo')\
  .size()\
  .sort_values(ascending=False)\
  .index[0]

'Poluição sonora'

In [6]:
#@title Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

chamados.merge(bairros, how="left", on="id_bairro")\
  .loc[chamados['data_inicio'].astype(str).str[:10] == '2023-04-01']\
  .groupby(['id_bairro', 'nome'])\
  .size()\
  .sort_values(ascending=False)\
  .head(3)

id_bairro  nome             
66         Engenho de Dentro    8
144        Campo Grande         6
26         Leblon               6
dtype: int64

In [7]:
#@title Qual o nome da subprefeitura com mais chamados abertos nesse dia?

chamados.merge(bairros, how="left", on="id_bairro")\
  .loc[chamados['data_inicio'].astype(str).str[:10] == '2023-04-01']\
  .groupby(['subprefeitura'])\
  .size()\
  .sort_values(ascending=False)\
  .index[0]

'Zona Norte'

In [8]:
#@title Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

chamados.merge(bairros, how="left", on="id_bairro")\
  .loc[
      (chamados['data_inicio'].astype(str).str[:10] == '2023-04-01') &
       (chamados['id_bairro'].isna() | bairros['subprefeitura'].isna())
       ]\
  .head()[['id_chamado', 'id_bairro', 'tipo', 'subtipo']]

,id_chamado,id_bairro,tipo,subtipo
44139,18516246,None,Ônibus,Verificação de ar condicionado inoperante no ô...


Existe um chamado sem bairro associado. Isso provavelmente se dá ao tipo e subtipo do chamado, que é uma verificação de ar condicionado inoperante de um Ônibus, logo, não é necessáriamente ligado a um bairro.



# Parte 2 - Chamados do 1746 em grandes eventos

In [9]:
#@title Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

chamados.loc[
    (chamados['subtipo'] == 'Perturbação do sossego') &
    (chamados['data_inicio'].astype(str).str[:10] >= '2022-01-01') &
    (chamados['data_inicio'].astype(str).str[:10] <= '2023-12-31')
    ]\
    .shape[0]

42408

In [10]:
#@title Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

# primeiro, vamos manipular os DFs para facilitar o join, a solução é ineficiente, o ideal seria realizar o join direto no sql

eventos['intervalo'] = list(zip(eventos['data_inicial'], eventos['data_final']))
eventos['evento_id'] = eventos.index

bins = pd.IntervalIndex.from_tuples(eventos['intervalo'], closed='both')

# criando correspondencias para os intervalos
categorias = pd.cut(chamados['data_inicio'], bins)

# criando coluna de evento
chamados['evento_id'] = categorias.cat.codes

chamados_eventos = chamados.merge(eventos, how='left', on='evento_id')

In [14]:
#agora, vamos selecionar apenas aqueles com subtipo pertubação de sossego e nos eventos listados
filtro = (
    (chamados_eventos['evento_id'] != -1) &
    (chamados_eventos['subtipo'] == 'Perturbação do sossego') &
    (chamados_eventos['evento'].isin(['Reveillon', 'Carnaval', 'Rock in Rio']))
)

chamados_eventos\
    .loc[filtro][['id_chamado', 'evento', 'data_inicial', 'subtipo', 'data_inicio']]

,id_chamado,evento,data_inicial,subtipo,data_inicio
10455,18078723,Reveillon,2022-12-30,Perturbação do sossego,2022-12-31 09:55:03
10458,18078416,Reveillon,2022-12-30,Perturbação do sossego,2022-12-31 00:38:53
10459,18078336,Reveillon,2022-12-30,Perturbação do sossego,2022-12-30 22:38:33
10460,18078403,Reveillon,2022-12-30,Perturbação do sossego,2022-12-31 00:08:42
10464,18078835,Reveillon,2022-12-30,Perturbação do sossego,2022-12-31 10:51:52
...,...,...,...,...,...
1597580,18329774,Carnaval,2023-02-18,Perturbação do sossego,2023-02-19 01:37:19
1597585,18329906,Carnaval,2023-02-18,Perturbação do sossego,2023-02-19 07:30:49
1597586,18330735,Carnaval,2023-02-18,Perturbação do sossego,2023-02-19 16:46:33
1597587,18330400,Carnaval,2023-02-18,Perturbação do sossego,2023-02-19 12:32:41


In [13]:
#@title Quantos chamados desse subtipo foram abertos em cada evento?

chamados_eventos\
    .loc[filtro]\
    .groupby(by=['evento', 'data_inicial'])\
    .size()\
    .sort_values(ascending=False)

evento       data_inicial
Rock in Rio  2022-09-08      306
             2022-09-02      212
Carnaval     2023-02-18      197
Reveillon    2022-12-30       79
dtype: int64

In [44]:
#@title Qual evento teve a maior média diária de chamados abertos desse subtipo?

df_filtrado = chamados_eventos.loc[filtro]
df_filtrado['duracao_evento'] = df_filtrado['data_final'] - df_filtrado['data_inicial'] + pd.Timedelta(days=1)


# Agrupa por 'evento' e 'data_inicial' e calcula o total de chamados
total_chamados_por_evento = df_filtrado.groupby(['evento', 'data_inicial']).agg({'id_chamado': 'count', 'duracao_evento': 'first'}).reset_index()

# media_eventos = total_chamados_por_evento / dias_por_evento
total_chamados_por_evento['media'] = total_chamados_por_evento['id_chamado'] / total_chamados_por_evento['duracao_evento'].dt.days
total_chamados_por_evento

,evento,data_inicial,id_chamado,duracao_evento,media
0,Carnaval,2023-02-18,197,4 days,49.250000
1,Reveillon,2022-12-30,79,3 days,26.333333
2,Rock in Rio,2022-09-02,212,3 days,70.666667
3,Rock in Rio,2022-09-08,306,4 days,76.500000


In [25]:
#@title Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023

chamados['data_inicio_normalizada'] = chamados['data_inicio'].dt.date


filtro_periodo = chamados[
    (chamados['data_inicio'] >= '2022-01-01') &
     (chamados['data_inicio'] <= '2023-12-31') &
      (chamados['subtipo'] == 'Perturbação do sossego')]

media_total = filtro_periodo['id_chamado'].nunique() / (pd.to_datetime('2023-12-31').date().toordinal() - pd.to_datetime('2022-01-01').date().toordinal() + 1)
media_total


58.09315068493151

In [46]:
# comparação

total_chamados_por_evento['proporcao'] = total_chamados_por_evento['media'] / media_total
total_chamados_por_evento


,evento,data_inicial,id_chamado,duracao_evento,media,proporcao
0,Carnaval,2023-02-18,197,4 days,49.250000,0.847776
1,Reveillon,2022-12-30,79,3 days,26.333333,0.453295
2,Rock in Rio,2022-09-02,212,3 days,70.666667,1.216437
3,Rock in Rio,2022-09-08,306,4 days,76.500000,1.316851
